# A2B Streaming Demo (Refactored)
This notebook demonstrates the streaming runner and producer. Ensure Kafka is running and topic `moth_clickstream` exists.

In [ ]:
# Start producer (in another shell) using: bash scripts/run_producer.sh
# Start streaming job (in another shell) using: bash scripts/run_streaming.sh

from moth import kafka_producer, spark_utils, visualisation

print('Producer and streaming runner available via scripts/.')

# Example: consume a small batch into pandas (visualisation)
# df = visualisation.consume_to_dataframe('localhost:9092', 'for_visualisation', timeout_ms=5000, max_messages=100)
# display(df.head())
